In [1]:
from __future__ import print_function
from scipy.misc import imsave
import numpy as np
import time
import os
import h5py

#import os #change theano device to cpu (it is gpu as default)    
#os.environ['THEANO_FLAGS'] = "device=cpu"    
import theano
import sklearn

Using gpu device 0: GeForce GTX 960 (CNMeM is disabled, cuDNN not available)


In [2]:
theano.config.device

'gpu'

In [8]:
# load data
data = np.genfromtxt('train.csv', delimiter=',')
data = data[1:,:] # ignore header

X_train = data[:,1:]
Y_train = data[:,0:1]

In [16]:
img_height = 28
img_width = 28
num_labels = 10

## Average digit

In [9]:
#automate process for remaining digits
avg_digit_array = np.zeros([img_height,0])
for i in xrange(10):
    sorted_array = data[data[:,0] == i]
    sorted_array = sorted_array[:,1:]
    
    
    avg_array = np.sum(sorted_array,axis=0)
    avg_array = np.reshape(avg_array,(img_height,img_width))
    avg_array /= 255
    avg_digit_array = np.append(avg_digit_array,avg_array,axis=1)


## what would accuracy be just using these averaged digits as filters ?

In [11]:
prediction = np.zeros([42000,1])
temp_sum = np.zeros([num_labels,1])
                      
for i in xrange(42000):
    X_temp = np.reshape(X_train[i:i+1,:],[img_width,img_height])
    for j in xrange(10):
        temp = X_temp * avg_digit_array[0:img_width,img_height*j:img_width*(j+1)]
        temp_sum[j] = temp.sum()
        #filtered_digit_array = np.append(filtered_digit_array, temp, axis=1)
    prediction[i] = np.argmax(temp_sum)

# NOT TOO BAD! not great either

In [13]:
accuracy = sum((prediction.flatten() == Y_train.flatten()))/float(len(Y_train))
print(accuracy)

0.672261904762


## visualize filters applied to one of each digit (0-9)

In [14]:
#grab one example of each filter from X_train
X_sample = np.zeros([num_labels,img_height*img_width])
for i in xrange(10):
    #take first example that is == i. not super efficient method
    X_sample[i,:] = data[data[:,0] == i][0,1:]

    
sample_filter_array = np.zeros([img_height*num_labels, img_width*num_labels])    
sample_prediction = np.zeros([num_labels, 1])
temp_sum = np.zeros([num_lables, 1])
                      
for i in xrange(num_labels):
    X_temp = np.reshape(X_sample[i:i+1,:],[img_height, img_width])
    for j in xrange(10):
        temp = X_temp * avg_digit_array[0:img_height, img_width*j:img_height*(j+1)]
        temp_sum[j] = temp.sum()
        sample_filter_array[img_height*j:img_width*(j+1),img_height*i:img_width*(i+1)] = temp
    prediction[i] = np.argmax(temp_sum)

X_sample_array = np.reshape(X_sample,[img_height*num_labels, img_width])

reshape_avg_digit_array = np.zeros([img_height*num_labels, img_width])
for i in xrange():
        reshape_avg_digit_array[28*i:28*(i+1),:] = avg_digit_array[:,28*i:28*(i+1)] 
avg_digit_array_T = np.reshape(avg_digit_array,[28*10,28])


In [15]:
import numpy as np
import matplotlib.pyplot as plt 

#plt.figure(figsize=(10, 10))
f, (a0, a1, a2) = plt.subplots(1,3, gridspec_kw = {'width_ratios':[1, 1, 10]}, figsize=(15,15))

a0.set_title("Input")
a1.set_title("Filter")
a2.set_title("Output (row[i] is result of Input[i] passing though each filter)")

a0.axis('off')
a1.axis('off')
a2.axis('off')

a0.imshow(X_sample_array,cmap='Greys_r')
a1.imshow(reshape_avg_digit_array,cmap='Greys_r')
a2.imshow(sample_filter_array,cmap='Greys_r')
#f.savefig('avg_digit_filter.pdf')
plt.show()

